In [1]:
import numpy as np
import gurobipy as gp
import json

The cell below loads data from a json file, and adds it into dictionaries; you should spend a few minutes to wrap your head around the data-format here.

In [2]:
with open('netflix_data.json') as json_file:
    data = json.load(json_file)

num_shows = data['num_shows'] # integer, denotes the number of shows that are being discussed
num_households = data['num_households'] # integer, denotes the number of households that may subscribe
production_budget = data['production_budget'] # integer, denotes the budget for shows to be produced
costs = {} # has as key shows (int) and as values the cost (int) of producing them
household_activation_utilities = {} # has as key households (int) and as values the threshold (int) at which they subscribe
show_utilities = {} # has as key households (int) and as values the utility a household gets from a show (int) 
show_utilities_uhd = {} # has as key households (int) and as values the utility a household gets from a show in UHD (int) 
uhd_household_threshold = {} # has as key households (int) and as values the threshold (int) at which they subscribe in UHD (if they subscribe) 
demographic = {} # has as key households and as values an integer from 0 to 4
for j in range(num_shows):
    costs[j] = int(data['costs'][str(j)])
for i in range(num_households):
    household_activation_utilities[i] = int(data['household_activation_utilities'][str(i)])
    uhd_household_threshold[i] = int(data['uhd_household_threshold'][str(i)])
    demographic[i] = int(data['demographic'][str(i)])
    show_utilities[i], show_utilities_uhd[i] = {}, {}
    for j in range(num_shows):
        show_utilities[i][j] = int(data['show_utilities'][str(i)][str(j)])
        show_utilities_uhd[i][j] = int(data['show_utilities_uhd'][str(i)][str(j)])

Below we implement the very first IP from class.

In [3]:
m1 = gp.Model('netflix_1')

# We first create a binary variable for every household and every show
# Household: does this household subscribe
# Show: is this show being produced

produce_show_1 = {}
for j in range(num_shows):
    produce_show_1[j] = m1.addVar(vtype=gp.GRB.BINARY)
subscribe_home_1 = {}
for i in range(num_households):
    subscribe_home_1[i] = m1.addVar(vtype=gp.GRB.BINARY)

# Constraint 1: budget on show costs
total_cost = 0
# m.addConstr(sum(produce_show_1[j] * costs[j] 
#                 for j in range(num_households))
for j in range(num_shows):
    total_cost += produce_show_1[j] * costs[j]
cost_constraint = m1.addConstr(total_cost <= production_budget)    

# Constraint 2: households only subscribe if shows fulfill their utility bound
for i in range(num_households):
    house_utility = 0
    for j in range(num_shows):
        house_utility+= show_utilities[i][j] * produce_show_1[j]
    m1.addConstr(house_utility >= subscribe_home_1[i] * household_activation_utilities[i])
    
# Define objective by summing up the number of subscriptions    
number_subscriptions = 0
for i in range(num_households):
    number_subscriptions += subscribe_home_1[i]
m1.setObjective( number_subscriptions, gp.GRB.MAXIMIZE)

m1.optimize()

Using license file /Users/dfreund/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 501 rows, 530 columns and 3518 nonzeros
Model fingerprint: 0xae1caa67
Variable types: 0 continuous, 530 integer (530 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+02, 5e+02]
Found heuristic solution: objective 44.0000000
Presolve removed 202 rows and 202 columns
Presolve time: 0.01s
Presolved: 299 rows, 328 columns, 2345 nonzeros
Variable types: 0 continuous, 328 integer (328 binary)

Root relaxation: objective 2.704334e+02, 572 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  270.43337    0  157   44.00000  270.43337   515%     -    0s
H    0     0                     18

We get an objective of 195 subscribers by producing the following shows.

In [4]:
shows_to_produce = []
for j in range(num_shows):
    if (produce_show_1[j]).x>0: 
        shows_to_produce.append(j)   
print(shows_to_produce)

[1, 2, 7, 9, 14, 17, 18, 20, 21, 22, 23, 25, 27]


Just to answer the question from class what would happen if we had continuous variables instead, let's run that really quick:

In [5]:
m1lp = gp.Model('netflix_1_lp')

# We first create a binary variable for every household and every show
# Household: does this household subscribe
# Show: is this show being produced

produce_show_1lp = {}
for j in range(num_shows):
    produce_show_1lp[j] = m1lp.addVar(lb=0, ub=1)
subscribe_home_1lp = {}
for i in range(num_households):
    subscribe_home_1lp[i] = m1lp.addVar(lb=0, ub=1)

# Constraint 1: budget on show costs
total_cost = 0
for j in range(num_shows):
    total_cost += produce_show_1lp[j] * costs[j]
cost_constraint = m1lp.addConstr(total_cost <= production_budget)    

# Constraint 2: households only subscribe if shows fulfill their utility bound
for i in range(num_households):
    house_utility = 0
    for j in range(num_shows):
        house_utility+= show_utilities[i][j] * produce_show_1lp[j]
    m1lp.addConstr(house_utility >= subscribe_home_1lp[i] * household_activation_utilities[i])
    

# Define objective by summing up the number of subscriptions    
number_subscriptions = 0
for i in range(num_households):
    number_subscriptions += subscribe_home_1lp[i]
m1lp.setObjective( number_subscriptions, gp.GRB.MAXIMIZE)

m1lp.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 501 rows, 530 columns and 3518 nonzeros
Model fingerprint: 0xe99493b0
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+02, 5e+02]
Presolve removed 239 rows and 239 columns
Presolve time: 0.01s
Presolved: 262 rows, 291 columns, 2114 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.6042857e+02   3.703815e+02   0.000000e+00      0s
     181    3.1384314e+02   0.000000e+00   0.000000e+00      0s

Solved in 181 iterations and 0.01 seconds
Optimal objective  3.138431393e+02


Quite noticeably two things happen: (1) we solve (almost) instantaneously instead of taking 1.5--2 minutes, and (2) we get a much higher objective; 313 instead of 195. Can we use the LP-solution to determine which shows to produce? Let's look at all the shows that the LP solution gives a positive objective to and their resulting cost.

In [6]:
shows_to_produce = []
total_cost = 0
for j in range(num_shows):
    if (produce_show_1lp[j]).x>0: 
        shows_to_produce.append(j)   
        total_cost += costs[j]
print(total_cost)

528


This violates our budget; however, we could instead use only the shows to which the LP solution gives a value of 1; this is guaranteed to give a feasible solution (can you see why?).

In [7]:
shows_to_produce = []
total_cost = 0
for j in range(num_shows):
    if (produce_show_1lp[j]).x==1: 
        shows_to_produce.append(j)   
        total_cost += costs[j]
print(total_cost)

483


Furthermore, it turns out that this gives the exact same solution as the IP did.

In [8]:
shows_to_produce

[1, 2, 7, 9, 14, 17, 18, 20, 21, 22, 23, 25, 27]

Is this true in general? Can we always just round down and get the optimal solution? The answer is no; you can try to come up with an example of input data for this problem that demonstrates as much (e.g., remember the controller from the Knapsack example in class; this here is similar)! =)